<a href="https://colab.research.google.com/github/PatricRc/Finance_Dashboard/blob/main/Streamlit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q

In [ ]:
!pip install yfinance -q

In [ ]:
!pip install pycoingecko -q

In [ ]:
%%writefile app.py
import pandas_datareader.data as reader
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
import streamlit as st
from keras.models import load_model
import pandas as pd
import requests
import tweepy
import config #archivo creado con los tokens de acceso a twitter
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timedelta
cg = CoinGeckoAPI()


dashboards = st.sidebar.selectbox('Escoge un dashboard', ('Analisis financiero', 'Graficas', 'Tweepy', 'Calculadora', 'Graficos Comparativos', 'Modelo de predicción'))
st.title(dashboards)

auth = tweepy.OAuthHandler(config.TWITTER_CONSUMER_KEY, config.TWITTER_CONSUMER_SECRET)
auth.set_access_token(config.TWITTER_ACCESS_TOKEN, config.TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


if dashboards == 'Analisis financiero':
    st.subheader('Este es el dashboard con analisis financieros')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')

    start = st.date_input('Fecha inicial', value = pd.to_datetime('2021-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
      
    st.header('Tabla de datos de {}'.format(user_input))
    df2 = yf.download(user_input,start,end).dropna()
    st.dataframe(df2.tail(50))
      
    def relativeret(df):
        rel = df.pct_change()
        cumret = (1+rel).cumprod()-1
        cumret = cumret.fillna(0)
        return cumret

    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    
    st.header('Analisis descriptivo de {}'.format(user_input))
    st.write(df.describe())

if dashboards == 'Graficas':
    st.header('Este es el dashboard con las graficas')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')
    start = st.date_input('Fecha inicial', value = pd.to_datetime('2019-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
    
    def relativeret(df):
      rel = df.pct_change()
      cumret = (1+rel).cumprod()-1
      cumret = cumret.fillna(0)
      return cumret
    
    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    df2 = yf.download(user_input,start,end).dropna()
    
    st.subheader('Retorno acumulado de {}'.format(user_input))
    st.line_chart(df)
  
    st.subheader('Precio de cierre vs Tiempo de {}'.format(user_input))
    df2 = yf.download(user_input,start,end).dropna()
    fig = plt.figure(figsize= (12,6))
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)

    st.subheader('Precio de cierre vs Tiempo con 100MA de {}'.format(user_input))
    ma100 = df2['Adj Close'].rolling(100).mean()
    fig = plt.figure(figsize= (12,6))
    plt.plot(ma100)
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)

    st.subheader('Precio de cierre vs Tiempo con 200MA de {}'.format(user_input))
    ma100 = df2['Adj Close'].rolling(100).mean()
    ma200 = df2['Adj Close'].rolling(200).mean()
    fig = plt.figure(figsize= (12,6))
    plt.plot(ma100)
    plt.plot(ma200)
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)

if dashboards == 'Graficos Comparativos':
    tickers = ('TSLA', 'AMZN', 'FB', 'BTC-USD','ETH-USD', 'AAPL')
    dropdown = st.multiselect('Escoge tus activos', tickers)
    start = st.date_input('Fecha inicial', value = pd.to_datetime('2019-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
    
    def relativeret(df):
        rel = df.pct_change()
        cumret = (1+rel).cumprod()-1
        cumret = cumret.fillna(0)
        return cumret

    if len(dropdown) > 0:
        df = relativeret(yf.download(dropdown, start, end)['Adj Close'])
        st.subheader('Retorno acumulado de {}'.format(dropdown))
        st.line_chart(df)


if dashboards == 'Tweepy':
    st.subheader('Este es el dashboard con analisis de principales tweets')
    tweets = api.user_timeline('elonmusk')
    st.write(tweets.text)

if dashboards == 'Modelo de predicción':
    st.subheader('Predicción de precio usando modelo de ML')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')

    start = st.date_input('Fecha inicial', value = pd.to_datetime('2019-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
    
    def relativeret(df):
      rel = df.pct_change()
      cumret = (1+rel).cumprod()-1
      cumret = cumret.fillna(0)
      return cumret
    
    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    df2 = yf.download(user_input,start,end).dropna()
    df_entrenado = pd.DataFrame(df2['Adj Close'][0:int(len(df)*0.70)])
    df_prueba = pd.DataFrame(df2['Adj Close'][int(len(df)*0.70) : int(len(df))])
 
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0,1))

    df_entrenado_array = scaler.fit_transform(df_entrenado)
    # cargar modelo de ML
    model = load_model('modelo_keras.h5')

    # Prueba del modelo de ML
    pasados_100d = df_entrenado.tail(100)
    df_final = pasados_100d.append(df_prueba, ignore_index=True)
    input_df = scaler.fit_transform(df_final)

    x_prueba = []
    y_prueba = []

    for i in range(100,input_df.shape[0]):
        x_prueba.append(input_df[i-100: i])
        y_prueba.append(input_df[i,0])

    x_prueba, y_prueba = np.array(x_prueba), np.array(y_prueba)
    y_predicted = model.predict(x_prueba)
    scaler = scaler.scale_

    scale_factor = 1/scaler[0]
    y_predicted = y_predicted * scale_factor
    y_prueba = y_prueba * scale_factor

    st.subheader('Data Precio Prediccion')
    st.dataframe(y_predicted)

# Grafico del modelo ML
    st.subheader('Precio Prediccion vs Original')
    fig2 = plt.figure(figsize=(12,6))
    plt.plot(y_prueba, 'b', label='Precio original')
    plt.plot(y_predicted, 'g', label='Precio pronosticado')
    plt.xlabel('Año')
    plt.ylabel('Precio')
    plt.legend()
    st.pyplot(fig2)

if dashboards == 'Calculadora':
    st.subheader('Esta es la calculadora de ahorro crypto')
    st.write("""
    # Crypto FOMO Calculator
    Usa las opciones para elegir los parametros:
    - Fecha en la que quisieras haber comprado $BTC
    - Cantidad en $ que hubieras invertido 
    """)
    st.write('---')
    
    st.image('btc-nft.png', use_column_width=True)

    #cargar datos
    btc_current = cg.get_price(ids = 'bitcoin', vs_currencies='usd')['bitcoin']['usd']
    
    #Giving Choices
    st.write('''# Escoge fecha y monto $''')
    today = datetime.utcnow().date()
    previous_day = today - timedelta(days=1)
    HIST_DATE = st.date_input("Fecha: ", value=previous_day, min_value=datetime(2014,1,1), max_value=previous_day)
    ORG_USD = st.number_input("Cantidad en $: ", min_value=1, max_value=999999999)

    #Formateando datos de fecha
    HIST_DATE_REFORMAT = HIST_DATE.strftime("%d-%m-%Y")
    HIST_DATE_datetime = datetime.strptime(HIST_DATE_REFORMAT,"%d-%m-%Y")
    btc_historic = cg.get_coin_history_by_id(id = 'bitcoin', vs_currencies='usd', date=HIST_DATE_REFORMAT)['market_data']['current_price']['usd']
    
    btc_historic = round(btc_historic, 2)
    eth_historic = round(btc_historic, 2)

    st.write('''# Results''')
    st.write('''## Historic Analysis''')
    st.write("Hubieses comprando una cantidad inicial de: ***{:,.2f}*** $BTC".format(round((ORG_USD/btc_historic),5)))
    st.write("Al precio de ***{:,.9f}***".format(btc_historic))
    st.write(" ")

    st.write('''## Efectos presentes''')
    total_btc = ORG_USD/btc_historic
    current_USD = total_btc * btc_current
    perc_change = (current_USD - ORG_USD)/(ORG_USD)*100
    usd_diff = current_USD - ORG_USD

    st.write("That is currently worth: ***${:,.2f}***".format(round(current_USD,2)))
    st.write("Which is a percentage change of ***{:,.2f}%***".format(round(perc_change, 2),))

    if usd_diff == 0:
      st.write('''# No has perdido ni ganado''')
    elif usd_diff <= 0:
      st.write('''# Hubieses perdido''')
    else:
      st.write('''# Te perdiste de ganar''') 
      st.write('***${:,.2f}!!!***'.format(abs(round(usd_diff,2)),))

    ## Analisis de Ethereum
    st.write('''# Ahora veamos el analisis de Ethereum''')
    eth_current = cg.get_price(ids = 'ethereum', vs_currencies='usd')['ethereum']['usd']
    eth_historic = cg.get_coin_history_by_id(id = 'ethereum', vs_currencies='usd', date=HIST_DATE_REFORMAT)['market_data']['current_price']['usd']
    eth_historic = round(eth_historic, 2)

    st.write('''## Results''')
    st.write('''## Historic Analysis''')
    st.write("Hubieses comprando una cantidad inicial de: ***{:,.2f}*** $ETH".format(round((ORG_USD/eth_historic),2)))
    st.write("Al precio de ***{:,.2f}***".format(eth_historic))
    st.write(" ")

    st.write('''## Efectos presentes''')
    total_eth = ORG_USD/eth_historic
    current_USD2 = total_eth * eth_current
    perc_change2 = (current_USD2 - ORG_USD)/(ORG_USD)*100
    usd_diff2 = current_USD2 - ORG_USD

    st.write("That is currently worth: ***${:,.2f}***".format(round(current_USD,2)))
    st.write("Which is a percentage change of ***{:,.2f}%***".format(round(perc_change2, 2),))

    if usd_diff2 == 0:
      st.write('''# No has perdido ni ganado''')
    elif usd_diff2 <= 0:
      st.write('''# Hubieses perdido''')
    else:
      st.write('''# Te perdiste de ganar''') 
      st.write('***${:,.2f}!!!***'.format(abs(round(usd_diff2,2)),))    


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.188.230.12:8501

npx: installed 22 in 3.354s
your url is: https://purple-stingray-31.loca.lt
2021-11-28 15:40:25.392 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  3 of 3 completed
  Stopping...
^C
